In [1]:
import monai
import ignite
import os


from prostate158.utils import load_config
from prostate158.train import SegmentationTrainer
# from prostate158.report import ReportGenerator
# from prostate158.viewer import ListViewer
from prostate158.data import segmentation_dataloaders
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# 加载配置文件
config_path = 'config/southHP/anatomy_swinunetr_48.yaml'
model_pth_path = '/home/data1/skyous/prostate158_log/checkpoint/swinunetr_0.8233.pt'
config = load_config(config_path) # change to 'tumor.yaml' for tumor segmentation
monai.utils.set_determinism(seed=config.seed)
config.data.valid_csv = 'southHP_predict/all_v3.csv'
config.data.train_csv = 'southHP_predict/all_v3.csv'
config.data.test_csv = 'southHP_predict/all_v3.csv'
# config.out_dir = 'southHP_predict_config'


config.data.data_dir = '/home/data1/skyous/SouthHP_prostate/sort_crop'
config.data.cache_dir = './tmp/monai-cache'

# config.data.valid_csv = '/home/data1/skyous/prostate158/prostate158_test/test.csv'
print(config.data.valid_csv)

southHP_predict/all_v3.csv


In [11]:
test_loader=segmentation_dataloaders(
            config=config, 
            train=False, 
            valid=False, 
            test=True
        )

trainer=SegmentationTrainer(
    progress_bar=True, 
    early_stopping = True, 
    metrics = ["MeanDice", "HausdorffDistance", "SurfaceDistance"],
    save_latest_metrics = True,
    config=config
)
trainer.evaluate(checkpoint=model_pth_path)

/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)
/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)
/home/skyous/.conda/envs/timm/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
 

metrics saved to /home/skyous/git/prostate158/southHP_predict/nnformer/anatomyprostate15820240322_0927_nnformer_1/output


In [12]:
import pandas as pd
import os

output_dir = '/home/skyous/git/prostate158/southHP_predict/nnformer/anatomyprostate15820240322_0927_nnformer_1/output'
# 读取CSV文件
DSC_data = pd.read_csv(os.path.join(output_dir, 'MeanDice_raw.csv'))
HD_data = pd.read_csv(os.path.join(output_dir, 'HausdorffDistance_raw.csv'))
SD_data = pd.read_csv(os.path.join(output_dir, 'SurfaceDistance_raw.csv'))

output_ = '|'

for data in [DSC_data, HD_data, SD_data]:
    # 获取某一列的数值

    for item in ['class0', 'class1', 'mean']:
        column_values = data[item]

        # 计算平均值
        average = column_values.mean()
        if data is DSC_data:
            average = round(average, 4)
        else:
            average = round(average, 2)
        print(item, average)
        output_ += str(average) + '|'

output_ += output_dir + '|'
print(output_)

class0 0.752
class1 0.5074
mean 0.6297
class0 24.47
class1 39.01
mean 31.74
class0 5.56
class1 6.95
mean 6.26
|0.752|0.5074|0.6297|24.47|39.01|31.74|5.56|6.95|6.26|/home/skyous/git/prostate158/southHP_predict/nnformer/anatomyprostate15820240322_0927_nnformer_1/output|


: 

|3dunet|0.7712|0.5376|0.6544|22.27|37.2|29.74|4.96|5.33|5.14|southHP_predict/3dunet/output|
|3dresunet|0.7969|0.616|0.7064|20.45|33.91|27.18|4.15|4.93|4.54|southHP_predict/3dresunet/anatomyprostate15820240322_0923_3dunet_1/output|
|nnunet|0.752|0.5074|0.6297|24.47|39.01|31.74|5.56|6.95|6.26|/home/skyous/git/prostate158/southHP_predict/nnformer/anatomyprostate15820240322_0927_nnformer_1/output|
